In [1]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import folium
from folium import plugins
from branca.colormap import LinearColormap

def get_gps_coordinates(gene_file, gps_file, line_number):
    # Read gene.data and gps files
    with open(gene_file, 'r') as gene_f, open(gps_file, 'r') as gps_f:
        gene_lines = gene_f.readlines()
        gps_lines = gps_f.readlines()

    # Ensure line_number is within range
    if line_number < 0 or line_number >= len(gene_lines):
        raise ValueError(f"line_number {line_number} is out of range.")

    # Get the specific line from gene.data
    indices_line = gene_lines[line_number].strip()
    indices = list(map(int, indices_line.split()))

    # Ensure indices are within range of gps_lines
    for index in indices:
        if index < 0 or index >= len(gps_lines):
            raise ValueError(f"Index {index} is out of range for GPS data.")

    # Extract GPS coordinates based on indices
    gps_coordinates = []
    for index in indices:
        gps_line = gps_lines[index].strip()
        gps_coordinates.append(list(map(float, gps_line.split())))

    # Convert to matrix (n, 2)
    gps_matrix = np.array(gps_coordinates)
    
    return gps_matrix
def plot_route_on_map(gps_matrix, epsilon, line_number, output_folder):
    # Create a map centered around the average coordinates
    avg_lat = np.mean(gps_matrix[:, 0])
    avg_lon = np.mean(gps_matrix[:, 1])
    folium_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=13)

    # Create a linear color gradient for the route
    gradient = np.linspace(0, 1, len(gps_matrix))

    # Create a colormap
    colormap = LinearColormap(['blue', 'red'], vmin=0, vmax=1).to_step(index=gradient)

    # Add markers for each point with color gradient
    for i, (lat, lon) in enumerate(gps_matrix):
        folium.CircleMarker([lat, lon], radius=3, color=colormap(gradient[i])).add_to(folium_map)

    # Add the route as a polyline with color gradient
    for i in range(len(gps_matrix) - 1):
        start = gps_matrix[i]
        end = gps_matrix[i + 1]
        color = colormap(gradient[i])
        folium.PolyLine([start, end], color=color, weight=2.5, opacity=1).add_to(folium_map)

    # Add a color bar legend
    colormap.caption = 'Distance Travelled'
    folium_map.add_child(colormap)

    # Save the map to an HTML file
    output_file = os.path.join(output_folder, f'route_map_{epsilon}_{line_number}.html')
    folium_map.save(output_file)
    print(f"Map saved to {output_file}")

# Example usage
gene_file = '../epsilon20/gene.data'
gps_file = '../epsilon20/gps'
line_number = 1233  # Specify the line number you want to process (0-based index)
epsilon = 'epsilon20'
output_folder = '../Map_Trajectories'

gps_matrix = get_gps_coordinates(gene_file, gps_file, line_number)
plot_route_on_map(gps_matrix, epsilon, line_number, output_folder)




Map saved to ../Map_Trajectories\route_map_epsilon20_1233.html
